<center><h1><font color=yellow><i> Ozone - DNN </i></font></h1></center>

In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Data

Import Data

In [47]:
data=pd.read_csv('ozone_complet_preprocessed.csv', sep=',')
data.head()

,Unnamed: 0,date,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,maxO3l
0,0,1995-04-01,47.6,10.1,11.6,13.3,13.6,12.2,8.0,8.0,...,4.0,300.0,4.0,340.0,4.0,20.0,4.0,-3.4641,62.2,56.2
1,1,1995-04-02,56.2,9.5,9.4,13.8,17.4,16.3,8.0,8.0,...,2.0,180.0,3.0,110.0,1.0,350.0,2.0,0.0000,47.6,61.8
2,2,1995-04-03,61.8,3.6,8.0,16.8,21.5,20.2,4.0,5.0,...,2.0,340.0,1.0,170.0,2.0,170.0,3.0,-0.3420,56.2,50.8
3,3,1995-04-04,50.8,9.5,10.5,11.4,12.2,11.4,8.0,7.0,...,4.0,350.0,3.0,350.0,3.0,350.0,4.0,-0.5209,61.8,59.8
4,4,1995-04-05,59.8,9.8,10.8,13.8,14.3,13.3,8.0,7.0,...,2.0,280.0,1.0,320.0,3.0,350.0,4.0,-0.9848,50.8,53.4


In [48]:
df_ozone=data

In [49]:

# Supposons que 'df' est votre DataFrame contenant la colonne 'date'
# Convertir la colonne 'date' en type datetime si elle ne l'est pas déjà
df_ozone['date'] = pd.to_datetime(df_ozone['date'])

# Définir une date de référence
date_reference = pd.to_datetime('1995-04-01')

# Calculer le nombre de jours depuis la date de référence
df_ozone['days_since_reference'] = (df_ozone['date'] - date_reference).dt.days

# Afficher les résultats
print(df_ozone[['date', 'days_since_reference']])
df_ozone=df_ozone.drop(columns='date',axis=1)


           date  days_since_reference
0    1995-04-01                     0
1    1995-04-02                     1
2    1995-04-03                     2
3    1995-04-04                     3
4    1995-04-05                     4
...         ...                   ...
1385 2002-09-25                  2734
1386 2002-09-26                  2735
1387 2002-09-27                  2736
1388 2002-09-28                  2737
1389 2002-09-29                  2738

[1390 rows x 2 columns]


Features and output

In [50]:
df_ozone.drop('Unnamed: 0', axis=1, inplace=True)
y=df_ozone['maxO3l']
X=df_ozone.drop('maxO3l',axis=1)

In [51]:
X.head()

,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,Ne12,Ne15,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,days_since_reference
0,47.6,10.1,11.6,13.3,13.6,12.2,8.0,8.0,8.0,8.0,...,4.0,300.0,4.0,340.0,4.0,20.0,4.0,-3.4641,62.2,0
1,56.2,9.5,9.4,13.8,17.4,16.3,8.0,8.0,7.0,0.0,...,2.0,180.0,3.0,110.0,1.0,350.0,2.0,0.0000,47.6,1
2,61.8,3.6,8.0,16.8,21.5,20.2,4.0,5.0,2.0,2.0,...,2.0,340.0,1.0,170.0,2.0,170.0,3.0,-0.3420,56.2,2
3,50.8,9.5,10.5,11.4,12.2,11.4,8.0,7.0,7.0,7.0,...,4.0,350.0,3.0,350.0,3.0,350.0,4.0,-0.5209,61.8,3
4,59.8,9.8,10.8,13.8,14.3,13.3,8.0,7.0,8.0,8.0,...,2.0,280.0,1.0,320.0,3.0,350.0,4.0,-0.9848,50.8,4


In [52]:
X.describe()

,maxO3,T6,T9,T12,T15,T18,Ne6,Ne9,Ne12,Ne15,...,Vvit9,Vdir12,Vvit12,Vdir15,Vvit15,Vdir18,Vvit18,Vx,maxO3v,days_since_reference
count,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,...,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000,1390.000000
mean,84.928269,12.303096,16.388913,19.420806,20.346364,18.783153,4.998559,5.159942,5.337176,5.150576,...,3.732901,205.017999,4.508279,208.351332,4.782577,190.820734,4.392369,-0.747871,85.037059,1384.066906
std,23.142702,3.898741,4.127919,4.926189,5.269710,5.079061,2.908986,2.564151,2.315037,2.323004,...,1.872830,105.871965,2.056291,110.564767,2.063139,121.417902,1.765431,3.401020,23.066851,848.879846
min,33.200000,-1.600000,4.500000,3.900000,6.500000,4.400000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.392300,33.200000,0.000000
25%,68.250000,10.000000,13.800000,16.225000,16.800000,15.500000,2.000000,3.000000,4.000000,3.000000,...,2.000000,110.000000,3.000000,100.000000,3.000000,60.000000,3.000000,-3.064200,68.650000,531.250000
50%,81.000000,12.700000,16.700000,19.400000,20.400000,18.600000,6.000000,6.000000,6.000000,6.000000,...,4.000000,220.000000,4.000000,230.000000,5.000000,220.000000,4.000000,-1.026100,81.600000,1481.500000
75%,97.000000,15.100000,19.000000,22.600000,23.800000,22.000000,8.000000,7.000000,7.000000,7.000000,...,5.000000,300.000000,6.000000,300.000000,6.000000,290.000000,6.000000,1.500000,97.000000,2202.750000
max,173.000000,21.700000,28.600000,33.500000,35.500000,34.100000,9.000000,9.000000,8.000000,8.000000,...,12.000000,360.000000,12.000000,360.000000,16.000000,360.000000,12.000000,10.832900,173.000000,2738.000000


Split our data into **train data** (80% -- 3680 rows) and **test data** (921 rows). 

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

# DNN

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
import keras_tuner


## Model created manually 

In [62]:
model1 = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]], name='hidden_layer1'),
    layers.Dropout(0.3),  # Ajout d'une couche Dropout pour réduire le surajustement
    layers.Dense(32, activation='relu', name='hidden_layer2'),
    layers.Dropout(0.3),  # Ajout d'une couche Dropout pour réduire le surajustement
    layers.BatchNormalization(),
    
    layers.Dense(1, activation='linear', name='output_layer'),  # Changement de l'activation en 'sigmoid'
])

In [63]:
from keras.optimizers import Adam
model1.compile(
    optimizer=Adam(learning_rate=0.005),
    loss='mean_squared_error',
    metrics=['mae'],
)

In [64]:
hist = model1.fit(X_train,
                 y_train,
                 batch_size=32,
                 epochs=50,
                 validation_data=(X_test,y_test))

Epoch 1/50
28/28 [==============================] - 2s 12ms/step - loss: 7641.5405 - mae: 84.2955 - val_loss: 7677.4478 - val_mae: 84.9901
Epoch 2/50
28/28 [==============================] - 0s 4ms/step - loss: 7185.9033 - mae: 81.8764 - val_loss: 8402.1631 - val_mae: 89.4124
Epoch 3/50
28/28 [==============================] - 0s 4ms/step - loss: 6359.7065 - mae: 77.1005 - val_loss: 5748.1313 - val_mae: 73.4097
Epoch 4/50
28/28 [==============================] - 0s 4ms/step - loss: 5247.3164 - mae: 69.6046 - val_loss: 4635.1846 - val_mae: 65.3880
Epoch 5/50
28/28 [==============================] - 0s 3ms/step - loss: 3929.3494 - mae: 59.6659 - val_loss: 2541.3525 - val_mae: 46.8745
Epoch 6/50
28/28 [==============================] - 0s 3ms/step - loss: 2651.9111 - mae: 47.8780 - val_loss: 1718.4198 - val_mae: 37.5484
Epoch 7/50
28/28 [==============================] - 0s 4ms/step - loss: 1608.0226 - mae: 35.5618 - val_loss: 751.1838 - val_mae: 22.8837
Epoch 8/50
28/28 [================

In [65]:
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

predictions=model1.predict(X_test)
# Calcul du RMSE (Root Mean Squared Error)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

# Calcul du MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, predictions)
print(f"MAE: {mae}")

# Coefficient de détermination (R²)
r2 = r2_score(y_test, predictions)
print(f"R² Score: {r2}")

9/9 [==============================] - 0s 2ms/step
RMSE: 16.73729502245305
MAE: 12.912595584238176
R² Score: 0.45225908906597745


## Keras tuner 

In [66]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8)

In [68]:
from kerastuner.tuners import RandomSearch
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('units1', min_value=20, max_value=70, step=20),
                           activation='relu', input_shape=[24], name='hidden_layer1'))
    model.add(layers.Dropout(0.3))

    

    model.add(layers.Dense(units=hp.Int('units2', min_value=20, max_value=100, step=20),
                           activation='relu', name='hidden_layer2'))
    model.add(layers.Dropout(0.3))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(1, activation='linear', name='output_layer'))

    model.compile(
    optimizer=Adam(learning_rate=5e-3),
    loss='mean_squared_error',
    metrics=['mae'],
)

    return model

tuner = RandomSearch(
    build_model,
    objective='mae',
    max_trials=20,  # Choisissez le nombre d'essais que vous voulez
    executions_per_trial=2,  # Nombre d'exécutions par essai pour réduire la variabilité
    directory='my_dirOzone1',
    project_name='keras_tuner'
)

print('Le résumé des hyperparamètres et de leur espace de recherche :', tuner.search_space_summary())

# Lancer la recherche d'hyperparamètres en utilisant l'ensemble d'entraînement (X_train, y_train) et l'ensemble de validation (X_val, y_val)
tuner.search(X_train, y_train,
             epochs=50,
             validation_data=(X_val, y_val)) 

best_model = tuner.get_best_models(num_models=1)[0]  # Renvoie les meilleurs modèles
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]  # Renvoie les meilleurs hyperparamètres


Trial 15 Complete [00h 00m 33s]
mae: 13.228235721588135

Best mae So Far: 13.074625492095947
Total elapsed time: 00h 08m 49s
